# Setup

In [ ]:
!nvidia-smi

In [1]:
import torch
import os 


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if torch.cuda.is_available():
    print("CUDA is available. Working on GPU.")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available. Working on CPU.")

CUDA is available. Working on GPU.
GPU name: NVIDIA GeForce RTX 3070 Laptop GPU


In [ ]:
# !curl -LsSf https://astral.sh/uv/install.sh | sh
# !source $HOME/.local/bin/env

These should be pre-installed in a colab env

In [ ]:
# import torch
# import transformers
# import accelerate
# import peft

# print(torch.__version__)
# print(transformers.__version__)
# print(accelerate.__version__)
# print(peft.__version__)

In [ ]:
#!pip install -U pip --progress-bar off
# !pip install -q bitsandbytes --progress-bar off
# !pip install -qqq datasets --progress-bar off
# !pip install -qqq loralib --progress-bar off
# !pip install -qqq einops --progress-bar off

imports that we will need

In [2]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')
login(token=HF_TOKEN)

# Data

https://www.kaggle.com/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset

In [ ]:
with open("datasetQ&A.json") as json_file:
    data = json.load(json_file)

In [ ]:
pprint(data["questions"][0], sort_dicts=False)
pprint(data["questions"][2], sort_dicts=False)
pprint(data["questions"][3], sort_dicts=False)

In [ ]:
with open("dataset.json", "w") as json_file:
    json.dump(data["questions"], json_file)

In [ ]:
pd.DataFrame(data["questions"]).head()

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

#MODEL_NAME = "meta-llama/Llama-3.2-1B" # 1B
#MODEL_NAME = "meta-llama/Llama-3.2-3B" # bastardo il cane di
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct" # 3B

bnb_config = BitsAndBytesConfig( 
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.use_cache = False
#model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
def print_trainable_parameters (model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters ():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


In [19]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [21]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 24313856 || all params: 1827777536 || trainable%: 1.330241537665993


In [27]:
prompt = """Your role is to be a student answering questions related to information retrieval ONLY WITH THE NUMBER REGARDING THE CORRECT ANSWER.
You're provided with a question and four options. Answer ONLY with the number of the correct option.
Question : In the vector space model, what is cosine similarity used for?
1) To calculate document frequency
2) To compute the angle between query and document vectors
3) To normalize term frequency
4) To adjust document rank""".strip()
print(prompt)

Your role is to be a student answering questions related to information retrieval ONLY WITH THE NUMBER REGARDING THE CORRECT ANSWER.
You're provided with a question and four options. Answer ONLY with the number of the correct option.
Question : In the vector space model, what is cosine similarity used for?
1) To calculate document frequency
2) To compute the angle between query and document vectors
3) To normalize term frequency
4) To adjust document rank


In [25]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.1
generation_config.top_p = 1.0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 200,
  "pad_token_id": 128009,
  "temperature": 0.1
}

In [30]:
%%time
device = "cuda:0"
 
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Your role is to be a student answering questions related to information retrieval ONLY WITH THE NUMBER REGARDING THE CORRECT ANSWER.
You're provided with a question and four options. Answer ONLY with the number of the correct option.
Question : In the vector space model, what is cosine similarity used for?
1) To calculate document frequency
2) To compute the angle between query and document vectors
3) To normalize term frequency
4) To adjust document rank

Correct answer is: 2) To compute the angle between query and document vectors
CPU times: total: 906 ms
Wall time: 3.02 s


# Kaggle dataset

In [ ]:
data = load_dataset("json", data_files="dataset.json")
data

In [ ]:
data["train"][0]

In [ ]:
def generate_prompt(data_point):
    return f"""
    <human>: {data_point["question"]}
    <assistant>: {data_point["answer"]}
    """.strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    encoding = tokenizer(full_prompt, padding=True, truncation=True)
    return encoding

In [ ]:
data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)
data

# Training

In [ ]:
OUTPUT_DIR = "experiments"

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir experiments/runs --port 8080

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=300,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


model.config.use_cache = False
trainer.train()

In [ ]:
model.save_pretrained("trained-model")

# Load the model

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

if torch.cuda.is_available():
    print("CUDA is available. Working on GPU.")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available. Working on CPU.")

In [ ]:
# load the model from the disk in 'trained-model'

#MODEL_NAME = "meta-llama/Llama-3.2-1B"
MODEL_NAME = "meta-llama/Llama-3.2-3B"
model = AutoModelForCausalLM.from_pretrained("trained-model").to(DEVICE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 50
generation_config.temperature = 0.0
generation_config.top_p = 0.0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time

prompt = f"""
<human>: How can I create an account?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model. generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True) )

In [ ]:
def generate_response(question: str) -> str:
    prompt = f"""
        <human>: {question}
        <assistant>:
        """.strip()
    encoding = tokenizer(prompt, return_tensors="pt") .to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) : ].strip()

In [ ]:
prompt = """Your role is to be a student answering questions related to information retrieval. You're provided with a question and four options. Answer ONLY with the number of the correct option.
Question : In the vector space model, what is cosine similarity used for?
1) To calculate document frequency
2) To compute the angle between query and document vectors
3) To normalize term frequency
4) To adjust document rank"""


print(generate_response(prompt))